# SAPO Adaptive I/J: Starting from J=0 (No Swarm Sharing)

This notebook tests the adaptive algorithm's ability to **discover the value of swarm sharing**.

**Configuration:**
- **I+J=4** (total rollouts per round, varies dynamically)
- **Initial J=0** (starts with NO external rollouts, 100% local)
- **G=8** (completions per question)
- **Model**: GPT-2 (124M params)
- **Hardware**: 5 nodes (1 coordinator + 4 workers) on 1× A100 80GB
- **Algorithm**: Gradient-Based Adaptive I/J (Version 2 from ADAPTIVE_IJ_ALGORITHM.md)

**Purpose:** Test if adaptive algorithm can improve from suboptimal initialization.

**Expected Behavior:**
- J starts at 0 (I=4, J=0 - pure local training, like baseline)
- Algorithm should gradually **increase J** toward optimal (~2)
- Demonstrates algorithm can discover swarm value from scratch

**Default Mode: TESTING (10 rounds, ~6 minutes)**
- Quick validation to ensure adaptive algorithm works
- See Cell 2 to switch to PRODUCTION mode (2000 rounds, ~21 hours)

**Memory Usage:** ~30 GB peak VRAM (4 workers train, coordinator doesn't)

**Thesis Contribution:** Proves adaptive algorithm can **find** optimal ratio, not just maintain it

**Run AFTER:** 
- Baseline (EX12.14a) for comparison
- Adaptive J=2 start (EX12.15) to compare learning curves

## 1. Configuration

**This notebook uses ADAPTIVE I/J selection starting from J=0.**

**⚠️ IMPORTANT: Testing Mode Enabled by Default**
- Default: 10 rounds (~6 minutes) - validates adaptive algorithm
- Production: 2000 rounds (~21 hours) - full training with adaptation

**To switch to production mode:** Change `MAX_ROUNDS` in Cell 2 below.

In [ ]:
# SAPO Adaptive I/J Experiment Configuration
# Starting from J=0 (no swarm sharing) to test adaptation capability

# ============================================
# ADAPTIVE I/J CONFIGURATION
# ============================================
EXPERIMENT_NAME = 'sapo_gpt2_adaptive_ij_startJ0'

# Total samples per round (I + J = 4)
TOTAL_SAMPLES = 4            # Total rollouts per round
INITIAL_J = 0                # Starting J value (NO external rollouts)
INITIAL_I = TOTAL_SAMPLES - INITIAL_J  # Computed automatically (I=4)

# Adaptive algorithm parameters
ADAPTIVE_IJ_ENABLED = True   # Enable adaptive I/J selection
ADAPTATION_RATE = 0.1        # Learning rate (alpha) for gradient updates
BASELINE_ALPHA = 0.95        # EMA smoothing for reward baseline

# ============================================
# TRAINING MODE: TESTING (default) or PRODUCTION
# ============================================
# TESTING MODE (default): Quick validation ~6 minutes
MAX_ROUNDS = 10              # Testing: 10 rounds to verify adaptive algorithm

# PRODUCTION MODE: Full training ~21 hours
# Uncomment line below for production run:
# MAX_ROUNDS = 2000          # Production: Full 2000 rounds for convergence

# ============================================
# FIXED SETTINGS (same for all experiments)
# ============================================
NUM_NODES = 5                # Run 5 nodes (1 coordinator + 4 workers)
MODEL_NAME = 'gpt2'          # GPT-2 (124M params, fits memory)
NUM_GENERATIONS = 8          # G: Completions per question (like paper)
SEED = 42                    # For reproducibility

# Rollout Sharing Configuration
ROLLOUT_PUBLISH_FREQUENCY = 'stage'  # When to share rollouts
ROLLOUT_CLEANUP_ENABLED = True       # Enable cleanup to save space
ROLLOUT_KEEP_LAST_N_ROUNDS = 20      # Keep recent rollouts only
ROLLOUT_ARCHIVE_OLD = False          # Don't archive (saves space)

# Checkpoint Configuration
CHECKPOINT_INTERVAL = 100    # Save checkpoints every 100 rounds
MAX_STAGES = 1               # Stages per round (1=default)

# Optional: HuggingFace Token
HUGGINGFACE_TOKEN = None  # Set to your token or keep None

# ============================================
# DISPLAY CONFIGURATION
# ============================================
mode = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_time = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"SAPO Adaptive I/J (Start J=0) - {mode} MODE")
print("="*60)
print(f"✓ Mode: {mode}")
print(f"✓ Nodes: {NUM_NODES} (1 coordinator + 4 workers on single A100 80GB)")
print(f"✓ Model: {MODEL_NAME}")
print(f"✓ Experiment: {EXPERIMENT_NAME}")
print(f"✓ Max Rounds: {MAX_ROUNDS}")
print()
print("🔄 ADAPTIVE I/J CONFIGURATION:")
print(f"   Total Samples: {TOTAL_SAMPLES} (I + J)")
print(f"   Initial: I={INITIAL_I}, J={INITIAL_J} (NO swarm sharing)")
print(f"   Adaptation Rate (α): {ADAPTATION_RATE}")
print(f"   Baseline Smoothing: {BASELINE_ALPHA}")
print()
print(f"Expected VRAM: ~30 GB peak (4 workers train, coordinator doesn't)")
print(f"Expected Time: {estimated_time}")
print()

if mode == "TESTING":
    print("🧪 TESTING MODE ENABLED")
    print("   Quick validation run - verifies:")
    print("   ✓ Algorithm starts at J=0 (pure local training)")
    print("   ✓ J increases if swarm sharing helps")
    print("   ✓ Logs show gradient updates pushing J upward")
    print()
    print("   After validation succeeds, uncomment production line")
    print("   in this cell to run full 2000-round training.")
else:
    print("📊 PRODUCTION MODE - Adaptive I/J from J=0")
    print("   Expected behavior:")
    print("   - J starts at 0 (baseline equivalent)")
    print("   - Should increase toward J=2 (optimal from paper)")
    print("   - Demonstrates algorithm can DISCOVER swarm value")
    print()
    print("   🎯 KEY THESIS TEST: Can it find optimum from scratch?")

print("="*60)

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set base path (MUST BE SAME ACROSS ALL NODES)
GDRIVE_BASE_PATH = '/content/drive/MyDrive/rl-swarm'
os.makedirs(GDRIVE_BASE_PATH, exist_ok=True)

print(f"✓ Google Drive mounted at: {GDRIVE_BASE_PATH}")

## 3. System Setup & GPU Verification

In [ ]:
import torch

print("="*60)
print("GPU Verification")
print("="*60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Total VRAM: {total_memory:.1f} GB")
    print()
    
    required_memory = (NUM_NODES - 1) * 6.5 + 4  # 4 workers + coordinator
    print(f"Memory Requirements:")
    print(f"  Workers: {NUM_NODES - 1} × 6.5 GB = {(NUM_NODES - 1) * 6.5:.1f} GB")
    print(f"  Coordinator: ~4 GB (non-training)")
    print(f"  Total estimated: {required_memory:.1f} GB")
    print(f"  Available: {total_memory:.1f} GB")
    print(f"  Margin: {total_memory - required_memory:.1f} GB")
    print()
    
    if total_memory < required_memory:
        raise RuntimeError(f"Insufficient GPU memory: need {required_memory:.0f} GB, have {total_memory:.1f} GB")
    else:
        print(f"✅ Sufficient VRAM for {NUM_NODES} nodes")
else:
    raise RuntimeError("No GPU detected! Select A100 GPU runtime: Runtime > Change runtime type > A100 GPU")

## 4. Clone Repository & Install Dependencies

In [ ]:
%cd /content

# Remove existing directory if it exists
if os.path.exists('/content/rl-swarm'):
    print("Removing existing repository...")
    !rm -rf /content/rl-swarm

# Clone fresh copy
print("Cloning repository...")
!git clone https://github.com/Elrashid/rl-swarm.git /content/rl-swarm

# Change to repo directory
%cd /content/rl-swarm

# Verify clone worked
if not os.path.exists('requirements.txt'):
    raise FileNotFoundError("Repository clone failed - requirements.txt not found")

print("✓ Repository cloned successfully")
print()

# Install dependencies
print("Installing dependencies (this may take 3-5 minutes)...")
!pip install -q -r requirements.txt
!pip install -q 'protobuf>=4.25.0,<5.0'

# Verify installation
try:
    import reasoning_gym
    from rgym_exp.src.adaptive_ij import GradientAdaptiveIJ
    print()
    print("✓ Dependencies installed successfully")
    print("✓ reasoning-gym verified")
    print("✓ Adaptive I/J algorithm verified")
except ImportError as e:
    print()
    print("❌ ERROR: Installation failed!")
    print(f"   {e}")
    raise

## 5. Initialize Experiment

In [ ]:
from rgym_exp.utils.experiment_manager import init_experiment

# Initialize experiment structure in Google Drive
config_overrides = {
    'training.max_round': MAX_ROUNDS,
    'training.num_generations': NUM_GENERATIONS,
    'training.total_samples': TOTAL_SAMPLES,
    'training.initial_I': INITIAL_I,
    'training.initial_J': INITIAL_J,
    'training.adaptive_ij_enabled': ADAPTIVE_IJ_ENABLED,
    'training.adaptation_rate': ADAPTATION_RATE,
    'training.seed': SEED,
}

init_experiment(
    gdrive_base_path=GDRIVE_BASE_PATH,
    experiment_name=EXPERIMENT_NAME,
    config_overrides=config_overrides
)

print(f"✓ Experiment initialized: {EXPERIMENT_NAME}")
print(f"  Path: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}")
print(f"  Config: Initial I={INITIAL_I}, J={INITIAL_J} (total={TOTAL_SAMPLES})")
print(f"  Adaptive: Enabled with α={ADAPTATION_RATE}")
print()
print("🎯 Starting from J=0 - algorithm must discover swarm value!")

## 6. Launch 5-Node Adaptive Swarm (KEY CELL)

**Note:** All nodes start with J=0 (no swarm sharing). Algorithm will adapt J upward if beneficial.

In [ ]:
import subprocess
import time
from datetime import datetime

mode_label = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_duration = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"Launching {NUM_NODES}-Node Adaptive Swarm (Start J=0, {mode_label})")
print("="*60)
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Model: {MODEL_NAME}")
print(f"Initial Config: I={INITIAL_I}, J={INITIAL_J}, G={NUM_GENERATIONS}")
print(f"Adaptation: α={ADAPTATION_RATE} (will increase J if swarm helps)")
print(f"Rounds: {MAX_ROUNDS} ({estimated_duration})")
print(f"Hardware: All {NUM_NODES} nodes on single GPU (A100 80GB)")
print("="*60)
print()

processes = []
start_time = time.time()

for node_id in range(NUM_NODES):
    # Environment variables for this node
    env = os.environ.copy()
    env['NODE_ID'] = f'node_{node_id}'
    env['NODE_ROLE'] = 'coordinator' if node_id == 0 else 'worker'
    env['MODEL_NAME'] = MODEL_NAME
    env['NUM_TRAIN_SAMPLES'] = str(INITIAL_I)  # Starting I value
    env['NUM_TRANSPLANT_TREES'] = str(INITIAL_J)  # Starting J value (0)
    env['NUM_GENERATIONS'] = str(NUM_GENERATIONS)
    env['MAX_ROUNDS'] = str(MAX_ROUNDS)
    env['EXPERIMENT_NAME'] = EXPERIMENT_NAME
    env['GDRIVE_PATH'] = GDRIVE_BASE_PATH
    env['CUDA_VISIBLE_DEVICES'] = '0'
    env['SEED'] = str(SEED + node_id)
    
    # Adaptive I/J configuration
    env['ADAPTIVE_IJ_ENABLED'] = 'True'
    env['ADAPTIVE_IJ_ALPHA'] = str(ADAPTATION_RATE)
    env['ADAPTIVE_IJ_BASELINE_ALPHA'] = str(BASELINE_ALPHA)
    env['ADAPTIVE_IJ_INITIAL_J'] = str(INITIAL_J)
    
    # Rollout sharing configuration
    env['ROLLOUT_PUBLISH_FREQUENCY'] = ROLLOUT_PUBLISH_FREQUENCY
    env['ROLLOUT_CLEANUP_ENABLED'] = str(ROLLOUT_CLEANUP_ENABLED)
    env['ROLLOUT_KEEP_LAST_N_ROUNDS'] = str(ROLLOUT_KEEP_LAST_N_ROUNDS)
    env['ROLLOUT_ARCHIVE_OLD'] = str(ROLLOUT_ARCHIVE_OLD)
    env['CHECKPOINT_INTERVAL'] = str(CHECKPOINT_INTERVAL)
    env['MAX_STAGES'] = str(MAX_STAGES)
    
    if HUGGINGFACE_TOKEN:
        env['HUGGINGFACE_ACCESS_TOKEN'] = HUGGINGFACE_TOKEN
    
    # Launch process
    import sys
    process = subprocess.Popen(
        [sys.executable, '-m', 'rgym_exp.runner.swarm_launcher'],
        env=env,
        cwd='/content/rl-swarm',
        stderr=subprocess.PIPE,
        stdout=subprocess.PIPE,
        text=True
    )
    processes.append(process)
    
    role = "COORDINATOR" if node_id == 0 else "WORKER     "
    print(f"✓ Started node_{node_id} ({role}) - PID: {process.pid:5d}")
    
    delay = 10 if node_id == 0 else 5
    time.sleep(delay)

print()
print("="*60)
print("✅ All nodes launched with J=0 start!")
print("="*60)
print()
print(f"✓ Training will run for {estimated_duration} ({MAX_ROUNDS} rounds)")
print(f"✓ J starts at 0, will adapt if swarm sharing helps")
print(f"✓ Logs location: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/")
print()
print("Monitor adaptive behavior in Cell 7 below...")

## 7. Monitor Adaptive Training Progress

**Watch J increase from 0 as algorithm discovers swarm value!**

In [ ]:
from IPython.display import clear_output
import pandas as pd

print("Starting adaptive training monitor...\n")

try:
    while True:
        clear_output(wait=True)
        
        running = sum(1 for p in processes if p.poll() is None)
        completed = NUM_NODES - running
        
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        elapsed_hours = (time.time() - start_time) / 3600
        
        print("="*70)
        print(f" Adaptive I/J Training Monitor (Start J=0) - {EXPERIMENT_NAME}")
        print(f" Time: {current_time} | Elapsed: {elapsed_hours:.1f}h")
        print("="*70)
        print()
        print(f"Nodes: {running}/{NUM_NODES} running, {completed} completed")
        print()
        
        # GPU memory
        if torch.cuda.is_available():
            reserved = torch.cuda.memory_reserved(0) / 1e9
            total = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"GPU: {reserved:.1f} / {total:.1f} GB ({reserved/total*100:.1f}%)")
            print()
        
        # Progress and adaptive I/J metrics
        try:
            from rgym_exp.utils.experiment_manager import get_experiment_status, get_experiment_metrics
            status = get_experiment_status(GDRIVE_BASE_PATH, EXPERIMENT_NAME)
            
            if status:
                current_round = status.get('current_round', 0)
                progress_pct = (current_round / MAX_ROUNDS) * 100
                print(f"Round: {current_round:4d} / {MAX_ROUNDS} ({progress_pct:5.1f}%)")
                
                # ETA
                if current_round > 5:
                    eta_hours = ((MAX_ROUNDS - current_round) * elapsed_hours) / current_round
                    print(f"ETA: {eta_hours:.1f} hours")
                
                # Adaptive I/J metrics
                df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)
                if not df.empty:
                    # Try to get adaptive metrics
                    if 'adaptive_I' in df.columns:
                        latest = df.tail(1).iloc[0]
                        current_I = int(latest.get('adaptive_I', INITIAL_I))
                        current_J = int(latest.get('adaptive_J', INITIAL_J))
                        J_cont = float(latest.get('adaptive_J_continuous', INITIAL_J))
                        baseline = float(latest.get('adaptive_baseline', 0))
                        
                        print(f"\n🔄 Adaptive I/J Status:")
                        print(f"   Started: I=4, J=0 (NO swarm)")
                        print(f"   Current: I={current_I}, J={current_J}")
                        print(f"   J (continuous): {J_cont:.2f}")
                        print(f"   Change: {current_J - INITIAL_J:+d} ({(current_J - INITIAL_J) if INITIAL_J == 0 else (current_J - INITIAL_J) / INITIAL_J * 100:+.1f}%)")
                        print(f"   Reward baseline: {baseline:.4f}")
                    
                    # Rewards
                    cumulative = df['my_reward'].sum()
                    recent = df.tail(10)['my_reward'].mean()
                    print(f"\nRewards: Cumulative={cumulative:.2f}, Recent avg={recent:.2f}")
        except Exception as e:
            print(f"(Metrics loading: {e})")
        
        print("\n" + "-"*70)
        print("Press 'Stop' to halt | Next update in 60s")
        
        if running == 0:
            print("\n" + "="*70)
            print("✅ Adaptive training completed!")
            print("="*70)
            break
        
        time.sleep(60)

except KeyboardInterrupt:
    print("\n⚠️  Interrupted - terminating processes...")
    for p in processes:
        if p.poll() is None:
            p.terminate()
    time.sleep(5)
    for p in processes:
        if p.poll() is None:
            p.kill()
    print("✓ Processes terminated")

## 8. Analyze Adaptive Behavior

**See how J evolved from 0 during training!**

In [ ]:
from rgym_exp.utils.experiment_manager import get_experiment_metrics
import matplotlib.pyplot as plt
import numpy as np

print("="*70)
print(f"Adaptive I/J Results (Start J=0): {EXPERIMENT_NAME}")
print("="*70)
print(f"Model: {MODEL_NAME}")
print(f"Rounds: {MAX_ROUNDS}")
print(f"Initial: I={INITIAL_I}, J={INITIAL_J} (NO swarm sharing)")
print(f"Adaptation Rate: α={ADAPTATION_RATE}")
print()

df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

if not df.empty and 'adaptive_J' in df.columns:
    # Aggregate by round (take last value per round)
    round_df = df.groupby('round').last().reset_index()
    
    final_I = int(round_df['adaptive_I'].iloc[-1])
    final_J = int(round_df['adaptive_J'].iloc[-1])
    final_J_cont = round_df['adaptive_J_continuous'].iloc[-1]
    
    total_reward = df['my_reward'].sum()
    
    print("Final Adaptive State:")
    print(f"  I={final_I}, J={final_J} (J_continuous={final_J_cont:.2f})")
    print(f"  Total Reward: {total_reward:.2f}")
    print()
    
    # Statistics
    mean_J = round_df['adaptive_J'].mean()
    std_J = round_df['adaptive_J'].std()
    
    print("Adaptive Statistics:")
    print(f"  Mean J: {mean_J:.2f} ± {std_J:.2f}")
    print(f"  Started at: J={INITIAL_J}")
    print(f"  Ended at: J={final_J}")
    print(f"  Change: {final_J - INITIAL_J:+d} (from baseline to swarm)")
    print()
    
    # Compare to expected
    print("Expected vs Observed:")
    print("  Paper optimal: I=2, J=2 (+94% vs baseline)")
    print(f"  Adaptive found: I={final_I}, J={final_J}")
    if final_J >= 1:
        print(f"  ✅ Algorithm discovered swarm value (J increased from 0 to {final_J})")
    else:
        print(f"  ⚠️  Algorithm kept J near 0 (needs more rounds or higher α)")
    print()
    
    # Plot adaptive trajectory
    if MAX_ROUNDS >= 20:
        fig, axes = plt.subplots(2, 1, figsize=(12, 10))
        
        # Plot 1: J evolution from 0
        ax1 = axes[0]
        ax1.plot(round_df['round'], round_df['adaptive_J_continuous'], 
                label='J (continuous)', linewidth=2, alpha=0.7)
        ax1.plot(round_df['round'], round_df['adaptive_J'], 
                label='J (discrete)', marker='o', markersize=3, alpha=0.5)
        ax1.axhline(y=0, color='red', linestyle='--', 
                   label='Starting J=0 (baseline)')
        ax1.axhline(y=2, color='green', linestyle='--', 
                   label='Paper optimal J=2')
        ax1.set_xlabel('Round')
        ax1.set_ylabel('J (External Rollouts)')
        ax1.set_title('Adaptive Discovery of Swarm Value (Starting J=0)', fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(-0.5, 4.5)
        
        # Plot 2: Cumulative rewards
        ax2 = axes[1]
        for node_id in df['node_id'].unique():
            node_df = df[df['node_id'] == node_id].sort_values('round')
            ax2.plot(node_df['round'], node_df['my_reward'].cumsum(), 
                    label=node_id, alpha=0.7)
        ax2.set_xlabel('Round')
        ax2.set_ylabel('Cumulative Reward')
        ax2.set_title('Cumulative Rewards with Adaptive I/J (Start J=0)', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plot_path = f'{GDRIVE_BASE_PATH}/adaptive_ij_startJ0_results.png'
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        print(f"✓ Plot saved: {plot_path}")
        plt.show()
        
        # Interpretation
        print()
        print("="*70)
        print("INTERPRETATION:")
        print("="*70)
        if final_J >= 2:
            print("✅ STRONG SUCCESS: Algorithm increased J to optimal level!")
            print("   - Started with NO swarm sharing (J=0, baseline)")
            print("   - Discovered swarm value through gradient ascent")
            print("   - Converged near paper's optimal J=2")
            print()
            print("📊 THESIS CONTRIBUTION:")
            print("   Adaptive algorithm can FIND optimal I/J ratio from scratch!")
        elif final_J >= 1:
            print("✅ PARTIAL SUCCESS: Algorithm increased J but not to optimal")
            print("   - Started at J=0, increased to J={final_J}")
            print("   - Recognized swarm value but needs more rounds")
            print("   - Or increase α for faster convergence")
            print()
            print("📊 THESIS INSIGHT:")
            print("   Algorithm is on right track - extend training for full convergence.")
        else:
            print("⚠️  LIMITED MOVEMENT: J stayed near 0")
            print("   Possible reasons:")
            print("   - Too few rounds for convergence (try 2000)")
            print("   - α too low (try 0.2 or 0.3)")
            print("   - Baseline reward not stabilized yet")
            print("   - Early training phase benefits local exploration")
        print()
else:
    print("❌ No adaptive metrics found - verify ADAPTIVE_IJ_ENABLED=True")